In [1]:
from helper import get_completion

prompt = "Write a poem about the ocean."
completion = get_completion(prompt)
print(completion)

The ocean, vast and deep
A world of wonder, secrets to keep
Its waves crash against the shore
A symphony of sound, forevermore

The salty air, the gentle breeze
Whispers of tales from distant seas
The sun sets, painting the sky
In hues of orange, pink, and gold up high

Beneath the surface, a world unseen
Where creatures of all shapes and sizes convene
From tiny fish to mighty whales
The ocean's beauty never fails

Its power, its mystery, its endless expanse
Captivates all who dare to dance
In its waves, in its embrace
The ocean's beauty, its grace

So let us cherish, protect, and adore
This wondrous world forevermore
For the ocean, with all its might
Is a treasure to behold, a true delight.


In [2]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader

# Load documents
documents = SimpleDirectoryReader(input_files=["./datasets/lora.pdf"]).load_data()

c:\Users\Admin\AppData\Local\pypoetry\Cache\virtualenvs\rags-agent-h2EaaEnM-py3.12\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [7]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [8]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

In [9]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [10]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document discusses the LoRA method, which involves freezing pre-trained model weights and incorporating trainable rank decomposition matrices into each layer of the Transformer architecture to efficiently adapt large language models for specific tasks. By reducing the number of trainable parameters, LoRA addresses challenges associated with full fine-tuning, such as high costs and storage requirements. Through empirical investigations, the document demonstrates that LoRA can outperform or match traditional fine-tuning methods while decreasing GPU memory requirements and training throughput. Various experiments and analyses on adaptation methods in deep learning models like GPT-2 and GPT-3 are detailed, focusing on language understanding and natural language generation tasks. The document explores the impact of hyperparameters, rank values, adaptation techniques, model layer correlations, and the amplif

In [11]:
print(len(response.source_nodes))

38
